# Long Short-Term Memory
### Project 2 - Data Science
- Abner Gardia 21285
- Esteban Donis 21610
- Daniel Gomez 21429

In [4]:
# Bloque 1: Importar librerías y cargar dataset
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dropout, Bidirectional
from transformers import TFAutoModel, AutoTokenizer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

Con la data limpia, procedemos a preparar los modelos de LSTM

In [5]:
discourses = pd.read_csv('./result/data_cleaned.csv')
discourses.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,text_length,word_count
0,0013cc385424,007ACE74B050,hi isaac going writing face mars natural landf...,4,0,164,27
1,9704a709b505,007ACE74B050,perspective think face natural landform nott t...,5,0,107,15
2,c22adee811b6,007ACE74B050,think face natural landform life mars descover...,0,0,52,8
3,a10d361e54e4,007ACE74B050,life mars would know reason think natural land...,3,0,199,30
4,db3e453ec4e2,007ACE74B050,people thought face formed alieans thought lif...,2,0,52,8


In [6]:
label_encoder = LabelEncoder()
discourses['discourse_effectiveness'] = label_encoder.fit_transform(discourses['discourse_effectiveness'])

X = discourses['discourse_text'].values
y = discourses['discourse_effectiveness'].values

X = [str(text) if not isinstance(text, str) else text for text in X]

tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(sequences, padding='post', maxlen=200)

X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

In [7]:
# LSTM Model 1: Simple LSTM
model1 = tf.keras.Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=200),
    LSTM(128),
    Dense(3, activation='softmax')
])
model1.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [8]:
model1.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))
model1.evaluate(X_test, y_test)

Epoch 1/5
920/920 ━━━━━━━━━━━━━━━━━━━━ 80s 86ms/step - accuracy: 0.5633 - loss: 0.9844 - val_accuracy: 0.5777 - val_loss: 0.9672
Epoch 2/5
920/920 ━━━━━━━━━━━━━━━━━━━━ 80s 86ms/step - accuracy: 0.5644 - loss: 0.9844 - val_accuracy: 0.5780 - val_loss: 0.9670
Epoch 3/5
920/920 ━━━━━━━━━━━━━━━━━━━━ 79s 86ms/step - accuracy: 0.5724 - loss: 0.9718 - val_accuracy: 0.5781 - val_loss: 0.9686
Epoch 4/5
920/920 ━━━━━━━━━━━━━━━━━━━━ 79s 86ms/step - accuracy: 0.5725 - loss: 0.9712 - val_accuracy: 0.5785 - val_loss: 0.9662
Epoch 5/5
920/920 ━━━━━━━━━━━━━━━━━━━━ 83s 90ms/step - accuracy: 0.5715 - loss: 0.9733 - val_accuracy: 0.5781 - val_loss: 0.9669
230/230 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - accuracy: 0.5779 - loss: 0.9695


[0.9668645262718201, 0.578131377696991]

In [9]:
# LSTM Model 2: Bidirectional LSTM
model2 = tf.keras.Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=200),
    Bidirectional(LSTM(128)),
    Dense(3, activation='softmax')
])
model2.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [10]:
model2.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))
model2.evaluate(X_test, y_test)

Epoch 1/5
920/920 ━━━━━━━━━━━━━━━━━━━━ 146s 158ms/step - accuracy: 0.6114 - loss: 0.8884 - val_accuracy: 0.6449 - val_loss: 0.8098
Epoch 2/5
920/920 ━━━━━━━━━━━━━━━━━━━━ 153s 166ms/step - accuracy: 0.6740 - loss: 0.7395 - val_accuracy: 0.6517 - val_loss: 0.7950
Epoch 3/5
920/920 ━━━━━━━━━━━━━━━━━━━━ 152s 165ms/step - accuracy: 0.7225 - loss: 0.6480 - val_accuracy: 0.6401 - val_loss: 0.8203
Epoch 4/5
920/920 ━━━━━━━━━━━━━━━━━━━━ 154s 167ms/step - accuracy: 0.7734 - loss: 0.5603 - val_accuracy: 0.6257 - val_loss: 0.8846
Epoch 5/5
920/920 ━━━━━━━━━━━━━━━━━━━━ 153s 167ms/step - accuracy: 0.8104 - loss: 0.4782 - val_accuracy: 0.6189 - val_loss: 0.9712
230/230 ━━━━━━━━━━━━━━━━━━━━ 12s 53ms/step - accuracy: 0.6120 - loss: 0.9687


[0.9711745977401733, 0.6189310550689697]

In [11]:
# LSTM Model 3: LSTM with Dropout
model3 = tf.keras.Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=200),
    LSTM(128, return_sequences=True),
    Dropout(0.5),
    LSTM(64),
    Dense(3, activation='softmax')
])
model3.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [12]:
model3.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))
model3.evaluate(X_test, y_test)

Epoch 1/5
920/920 ━━━━━━━━━━━━━━━━━━━━ 133s 144ms/step - accuracy: 0.5634 - loss: 0.9855 - val_accuracy: 0.5775 - val_loss: 0.9794
Epoch 2/5
920/920 ━━━━━━━━━━━━━━━━━━━━ 125s 136ms/step - accuracy: 0.5724 - loss: 0.9763 - val_accuracy: 0.5775 - val_loss: 0.9694
Epoch 3/5
920/920 ━━━━━━━━━━━━━━━━━━━━ 128s 140ms/step - accuracy: 0.5663 - loss: 0.9773 - val_accuracy: 0.5775 - val_loss: 0.9677
Epoch 4/5
920/920 ━━━━━━━━━━━━━━━━━━━━ 127s 138ms/step - accuracy: 0.5657 - loss: 0.9790 - val_accuracy: 0.5775 - val_loss: 0.9674
Epoch 5/5
920/920 ━━━━━━━━━━━━━━━━━━━━ 130s 141ms/step - accuracy: 0.5685 - loss: 0.9749 - val_accuracy: 0.5775 - val_loss: 0.9663
230/230 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.5773 - loss: 0.9687


[0.9662852883338928, 0.5774514079093933]